In [1]:
import os
import sys
import gc

import random
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cross_validation import train_test_split
from sklearn.metrics import accuracy_score
path = '/home/zhouzr/data/tianchi/user_location_predict/original_data/'
os.chdir(path)

train = pd.read_csv('./训练数据-ccf_first_round_user_shop_behavior.csv')
shop = pd.read_csv('./训练数据-ccf_first_round_shop_info.csv')
test = pd.read_csv('./AB榜测试集-evaluation_public.csv')
train = pd.merge(train, shop[['shop_id','mall_id']], on='shop_id', how='left')

mall_id_list = shop.mall_id.unique()
mall = mall_id_list[0]
train_mall = train[train.mall_id==mall]

/home/zhouzr/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [77]:
# latitude longitude knn top5 acc
knn = KNeighborsClassifier(n_neighbors=5)
train_x, test_x, train_y, test_y = train_test_split(train_mall[['longitude', 'latitude']], train_mall.shop_id)
knn.fit(train_x, train_y)

forcast = knn.predict_proba(test_x)
label_list = knn.classes_
top5forcast = []

for f in forcast:
    min_prob = max(np.sort(f)[-5], f[f>0].min())
    top_forcast_f = label_list[f>=min_prob]
    if len(top_forcast_f) < 5:
        top_forcast_f = np.append(top_forcast_f, ['None'] * (5-len(top_forcast_f)))
    top5forcast.append(list(top_forcast_f))

forcast_acc = np.array([])
for k, y in enumerate(test_y):
    if y in top5forcast[k]:
        forcast_acc = np.append(forcast_acc, 1)
    else:
        forcast_acc = np.append(forcast_acc, 0)

print(forcast_acc.mean())

0.80638424821


In [7]:
# latitude longitude knn top3 acc
knn = KNeighborsClassifier(n_neighbors=5)
train_x, test_x, train_y, test_y = train_test_split(train_mall[['longitude', 'latitude']], train_mall.shop_id)
knn.fit(train_x, train_y)

forcast = knn.predict_proba(test_x)
label_list = knn.classes_
top3forcast = []

for f in forcast:
    min_prob = max(np.sort(f)[-3], f[f>0].min())
    top_forcast_f = label_list[f>=min_prob]
    if len(top_forcast_f) < 3:
        top_forcast_f = np.append(top_forcast_f, ['None'] * (3-len(top_forcast_f)))
    top3forcast.append(list(top_forcast_f))

forcast_acc = np.array([])
for k, y in enumerate(test_y):
    if y in top3forcast[k]:
        forcast_acc = np.append(forcast_acc, 1)
    else:
        forcast_acc = np.append(forcast_acc, 0)

print(forcast_acc.mean())

0.797881861575


In [11]:
# latitude longitude knn top1 acc
knn = KNeighborsClassifier(n_neighbors=5)
train_x, test_x, train_y, test_y = train_test_split(train_mall[['longitude', 'latitude']], train_mall.shop_id)
knn.fit(train_x, train_y)

forcast = knn.predict(test_x)
accuracy_score(test_y, forcast)

0.59531622911694515